# To-Do List

- [x] Explore different embeddings
- [x] Add better metadata to chunks
- [ ] Use graph-based rag for better retrieval
- [ ] Test RAG
- [ ] Automate testing

# Import Libraries

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader, LocalBaseLoader, LocalFolderLoader, LocalFileLoader, LocalFolderLoader
from langchain_community.vectorstores import Chroma
from groq import Groq
from langchain_groq import ChatGroq
from src.utils import *
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import os
from dotenv import load_dotenv

utils.py loaded


In [2]:
# import api keys
# .\qrag-env\Scripts\activate # command to activate virtual environment in terminal
load_dotenv()
groq_api_key = os.getenv("groq_api_key")
openai_api_key = os.getenv("chatgpt.gpt3.apiKey")
mistra_api_key = os.getenv("mistra_api_key")
tavily_api_key = os.getenv("tavily_api_key")

# Instantiate Embedding Model

The choice of the embedding model is important and depends on the RAG application. Since a specific LLM is trained using a particular embedding model, the choice of LLM in the RAG application dictates the choice of the embedding model. Here is a table of LLM's and their embedding models. 

| Model                        | Benchmark/Metric          | Performance Highlights                                                                 |
|------------------------------|---------------------------|----------------------------------------------------------------------------------------|
| SPECTER2                     | SciRepEval                | Outperforms general-purpose models on regression, classification, and re-ranking tasks. |
| Snowflake Arctic Embed       | MTEB Retrieval Leaderboard| Ranks first among models of similar size; large model outperforms models with 1B+ parameters. |
| Tabular Embedding Model (TEM)| Domain-Specific Performance| Excels in handling high-dimensional datasets, particularly in financial and scientific domains. |
| Voyage AI Embedding Models   | RAG Implementation        | High retrieval quality and efficiency in generating accurate responses[4].              |
| OpenAI text-embedding-3-large| Embedding Dimensions      | High performance in multi-language retrieval and task-specific accuracy[11].            |


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_mistralai import MistralAIEmbeddings

openai = OpenAIEmbeddings(model='text-embedding-3-large',api_key=openai_api_key)
mistral = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=mistra_api_key,
)
# embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

c:\Users\evang\Desktop\GraduateSchool\Graduation\qRAG\qrag-env\Lib\site-packages\langchain_mistralai\embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


# Instantiate LLM

In [4]:
llm = ChatGroq(temperature=0,
              # model_name="Llama3-70b-8192",
              model_name="mixtral-8x7b-32768",
              api_key=groq_api_key)

# Import Data (Theses, Papers)

Since the data consists of pdf documents, they first need to be parsed, split into chunks, and then embedded. There are multiple packages that can be used to load and parse pdf's, like pdflumber, pypdf, and pymupdf. It seems like pymypdf is the best according to the online literature. Also, pymupdf seems to have extensive documentation and support, unlike pdflumber, including examples of using pymupdf for RAG applications. Neat! 

If you want to learn more, [https://pymupdf.readthedocs.io/en/latest/index.html](URL)

In [5]:
theses_paths = get_file_paths(r"Theses/")
papers_paths = get_file_paths(r"Papers/")

Theses\intro-to-quantum-measurement.pdf
Theses\Mivev-Thesis.pdf
Theses\Schuster-Thesis.pdf
Theses\Schwarz-Thesis.pdf
Theses\Slichter-Thesis.pdf
Papers\oliver-QME.pdf
Papers\qe-guide-scqubits-oliver.pdf
Papers\wendin-review-scqubits.pdf


In [ ]:
from langchain_community.document_loaders import DirectoryLoader
theses = load_pdf_document(theses_paths)
papers = load_pdf_document(papers_paths)

## Save markdown

In [ ]:
theses[0]

In [6]:
save_markdown_text(theses,theses_paths)
save_markdown_text(papers,papers_paths)

NameError: name 'theses' is not defined

## Load Markdown Documents

In [8]:
md_theses = load_markdown_text(theses_paths)
md_papers = load_markdown_text(papers_paths)

Loading markdown text from: Theses\intro-to-quantum-measurement.md
Loading markdown text from: Theses\Mivev-Thesis.md
Loading markdown text from: Theses\Schuster-Thesis.md
Loading markdown text from: Theses\Schwarz-Thesis.md
Loading markdown text from: Theses\Slichter-Thesis.md
Loading markdown text from: Papers\oliver-QME.md
Loading markdown text from: Papers\qe-guide-scqubits-oliver.md
Loading markdown text from: Papers\wendin-review-scqubits.md


## Split into chunks

Documents are split into chunks depending on the headers. Useful metadata such as chunk type (chapter, section, subsection, etc.) and name are created. Further splitting according to meaning is done after that to enchance chunking and group chunks according to similarity in embedding space. This method is better compared to brute-force splitting where the only input parameters are the chunk_size and chunk_overlap. Keep in mind that the final chunk size needs to conform to the size of the embedding model. For more information, here are some useful links.

- **Semantic Splitting with Langchain**: https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/
- **Information on how to choose the right embedding model**: https://datasciencedojo.com/blog/choosing-the-right-vector-embedding-model/

In [9]:
# Splitting the documents into chunks based on the headers
headers_to_split_thesis_on = [
    ("#", "Chapter"),
    ("##", "Section"),
    ("###", "Subsection"),
    ("####", "Subsubsection"),
    ("#####", "Subsubsubsection"),
    # ("#"*6, "Subsubsubsubsection")
]

theses_chunks = split_docs(md_theses, headers_to_split_thesis_on)
papers_chunks = split_docs(md_papers, headers_to_split_thesis_on)
chunks = theses_chunks + papers_chunks

5 documents split into 227 chunks
3 documents split into 35 chunks


**Further Chunking Based on Meaning (Not functional yet)**

In [ ]:
# Splitting documents based on semantic similarity
from langchain_experimental.text_splitter import SemanticChunker

semantic_text_splitter = text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile",
)
docs = semantic_text_splitter.create_documents(papers_chunks)

# Vector Database

In [10]:
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=openai,
                                    collection_name="local-rag")

# Setup Agents

In [11]:
from agents import crew, agent_prompts
from langchain_community.tools.tavily_search import TavilySearchResults

agents = crew(agent_prompts,llm)
agents.build_crew()
web_search_tool = TavilySearchResults(k=3)

Building agent: question_router
Building agent: rag_chain
Building agent: retrieval_grader
Building agent: hallucination_grader
Building agent: answer_grader


# Create Graph

In [12]:
from graph import RAG_graph
from nodes import nodes

# actions = nodes(agents.crew, web_search_tool)
rag_graph = RAG_graph(agents.crew, web_search_tool, vectorstore)

# Compile & Test Workflow

In [13]:
from langchain_core.runnables import RunnableConfig
app = rag_graph.compile()
config = RunnableConfig(recursion_limit=3)

In [15]:
from pprint import pprint
inputs = {"question": "What are the main requirements for being able to detect quantum jumps?"}
for output in app.stream(inputs,config):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
What are the main requirements for being able to detect quantum jumps?
{'datasource': 'web_search'}
web_search
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH---
'Finished running: websearch:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---
'Finished running: generate:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---
'Finished running: generate:'


GraphRecursionError: Recursion limit of 3 reachedwithout hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.

# Display Graph

In [47]:
from IPython.display import Image

Image(app.get_graph().draw_png(output_file_path="graph"))

filename:  b'graph'.gv


Exception ignored in: <function AGraph.__del__ at 0x000001E5B71C7A60>
Traceback (most recent call last):
  File "c:\Users\evang\Desktop\GraduateSchool\Graduation\qRAG\qrag-env\Lib\site-packages\pygraphviz\agraph.py", line 296, in __del__
    self._close_handle()
  File "c:\Users\evang\Desktop\GraduateSchool\Graduation\qRAG\qrag-env\Lib\site-packages\pygraphviz\agraph.py", line 1013, in _close_handle
    gv.agclose(self.handle)
    ^^^^^^^^^^
AttributeError: module 'pygraphviz.graphviz' has no attribute 'agclose'


AttributeError: module 'pygraphviz.graphviz' has no attribute 'agraphof'

In [25]:
from IPython.display import Image

type(app.get_graph())

langgraph.utils.DrawableGraph